Importing the dependencies

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Preprocessing

In [4]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/news.csv')

In [5]:
news_dataset.shape

(6335, 4)

In [6]:
# print the first 5 rows of the dataframe
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [7]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [9]:
# merging the author name and news title
news_dataset['content'] = news_dataset['text']+' '+news_dataset['title']

In [10]:
print(news_dataset['content'])

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [11]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [12]:
print(X)
print(Y)

      Unnamed: 0  ...                                            content
0           8476  ...  Daniel Greenfield, a Shillman Journalism Fello...
1          10294  ...  Google Pinterest Digg Linkedin Reddit Stumbleu...
2           3608  ...  U.S. Secretary of State John F. Kerry said Mon...
3          10142  ...  — Kaydee King (@KaydeeKing) November 9, 2016 T...
4            875  ...  It's primary day in New York and front-runners...
...          ...  ...                                                ...
6330        4490  ...  The State Department told the Republican Natio...
6331        8062  ...  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332        8622  ...   Anti-Trump Protesters Are Tools of the Oligar...
6333        4021  ...  ADDIS ABABA, Ethiopia —President Obama convene...
6334        4330  ...  Jeb Bush Is Suddenly Attacking Trump. Here's W...

[6335 rows x 4 columns]
0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [15]:
print(news_dataset['content'])

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [16]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [17]:
print(X)


['Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is being warned that the enti

In [18]:
print(Y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [20]:
Y.shape

(6335,)

In [21]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [22]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2172758 stored elements and shape (6335, 67926)>
  Coords	Values
  (0, 1629)	0.01620067894685332
  (0, 2319)	0.02091617172019117
  (0, 2412)	0.012362968166478416
  (0, 2501)	0.024083334170135
  (0, 2651)	0.016015097660191803
  (0, 2655)	0.04332786565935788
  (0, 2764)	0.05119995295824521
  (0, 2994)	0.022388927309660397
  (0, 3051)	0.020332760645694843
  (0, 3232)	0.059975065138012595
  (0, 3252)	0.007110813516507457
  (0, 3274)	0.03356572352573969
  (0, 3283)	0.016211225437576967
  (0, 3292)	0.015386662303415337
  (0, 3296)	0.03629743475813936
  (0, 3348)	0.011742047788665014
  (0, 3394)	0.014483350138037688
  (0, 3768)	0.020198388339288345
  (0, 3780)	0.032391440169600466
  (0, 3792)	0.01929395208818379
  (0, 3831)	0.030721089197861658
  (0, 3853)	0.015232262979818117
  (0, 3919)	0.02182152114431929
  (0, 3924)	0.013725073522225474
  (0, 4085)	0.018099493273326585
  :	:
  (6334, 65088)	0.020695591830443796
  (6334, 65097)	

Splitting the dataset to training & test data

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [24]:
model = LogisticRegression()

In [25]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [27]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [28]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9516574585635359


In [29]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [30]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9100236779794791


Making a Predictive System

In [52]:
X_new = X_test[6]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]=='REAL'):
  print('The news is real')
else:
  print('The news is fake')


['FAKE']
The news is fake


In [53]:
print(Y_test[6])

FAKE
